In [21]:
import pandas as pd
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import accuracy_score, mean_absolute_error, mean_squared_error, r2_score
from sklearn.preprocessing import LabelEncoder

In [23]:
# 📌 1️⃣ VERİYİ YÜKLEME
electricity_data = pd.read_csv("electricity-data.csv")

In [25]:
electricity_data.head()

,date,hour,price,priceUsd,priceEur,toplam,dogalgaz,ruzgar,linyit,tasKomur,...,HAM-10,HAM-20,RWE,TEKIRA,TEKIRB,YENI,IST A-(A),IST A-(B),IST A-(C),IST B (Blok40+ Blok50)
0,2020-01-01T00:00:00+03:00,00:00,311.65,52.46,46.86,25725.50,5646.60,849.37,5331.5,276.0,...,0.0,463.0,101.3,200,100,0.0,0,0,0,799
1,2020-01-01T01:00:00+03:00,01:00,313.24,52.73,47.10,24159.99,4731.56,826.17,5331.5,276.0,...,0.0,468.0,0.0,0,0,0.0,0,0,0,772
2,2020-01-01T02:00:00+03:00,02:00,311.08,52.37,46.77,22937.15,4535.73,798.83,5306.5,276.0,...,0.0,455.0,0.0,0,0,0.0,0,0,0,590
3,2020-01-01T03:00:00+03:00,03:00,310.70,52.30,46.72,21972.48,3322.21,797.41,5249.0,276.0,...,0.0,436.0,0.0,0,0,0.0,0,0,0,520
4,2020-01-01T04:00:00+03:00,04:00,200.01,33.67,30.07,21451.33,2848.20,872.79,5360.6,0.0,...,0.0,220.0,0.0,0,0,0.0,0,0,0,480


In [27]:
# 1️⃣ **Veriyi Yükleme ve Feature Engineering**
electricity_data = pd.read_csv("electricity-data.csv")

electricity_data["date"] = pd.to_datetime(electricity_data["date"], utc=True, errors='coerce').dt.tz_convert(None)

electricity_data["year"] = electricity_data["date"].dt.year
electricity_data["month"] = electricity_data["date"].dt.month
electricity_data["day"] = electricity_data["date"].dt.day
electricity_data["day_of_week"] = electricity_data["date"].dt.weekday
electricity_data["is_weekend"] = (electricity_data["day_of_week"] >= 5).astype(int)
electricity_data["hour_numeric"] = electricity_data["hour"].str[:2].astype(int)

electricity_data["season"] = electricity_data["month"].map({12: "Winter", 1: "Winter", 2: "Winter",
                                                                3: "Spring", 4: "Spring", 5: "Spring",
                                                                6: "Summer", 7: "Summer", 8: "Summer",
                                                                9: "Autumn", 10: "Autumn", 11: "Autumn"})

In [29]:
electricity_data.shape

(43848, 62)

In [31]:
electricity_data.head()

,date,hour,price,priceUsd,priceEur,toplam,dogalgaz,ruzgar,linyit,tasKomur,...,IST A-(B),IST A-(C),IST B (Blok40+ Blok50),year,month,day,day_of_week,is_weekend,hour_numeric,season
0,2019-12-31 21:00:00,00:00,311.65,52.46,46.86,25725.50,5646.60,849.37,5331.5,276.0,...,0,0,799,2019.0,12.0,31.0,1.0,0,0,Winter
1,2019-12-31 22:00:00,01:00,313.24,52.73,47.10,24159.99,4731.56,826.17,5331.5,276.0,...,0,0,772,2019.0,12.0,31.0,1.0,0,1,Winter
2,2019-12-31 23:00:00,02:00,311.08,52.37,46.77,22937.15,4535.73,798.83,5306.5,276.0,...,0,0,590,2019.0,12.0,31.0,1.0,0,2,Winter
3,2020-01-01 00:00:00,03:00,310.70,52.30,46.72,21972.48,3322.21,797.41,5249.0,276.0,...,0,0,520,2020.0,1.0,1.0,2.0,0,3,Winter
4,2020-01-01 01:00:00,04:00,200.01,33.67,30.07,21451.33,2848.20,872.79,5360.6,0.0,...,0,0,480,2020.0,1.0,1.0,2.0,0,4,Winter


In [33]:
# 📌 GÜNCELLENMİŞ FEATURE SETİ
features = [
    "hour_numeric", "day_of_week", "is_weekend", "month", "season", 
    "ruzgar", "dogalgaz", "lep", "consumption", "barajli", "gunes"
]

In [39]:
print(electricity_data.columns.tolist())

['date', 'hour', 'price', 'priceUsd', 'priceEur', 'toplam', 'dogalgaz', 'ruzgar', 'linyit', 'tasKomur', 'ithalKomur', 'fuelOil', 'jeotermal', 'barajli', 'nafta', 'biokutle', 'akarsu', 'diger', 'lep', 'consumption', 'toplam_uga', 'ruzgar_uga', 'biyogaz', 'kanalTipi', 'biyokutle', 'gunes', 'diger_uga', 'ACWA', 'AKENRJ ERZIN', 'AKSA ANT', 'BAN1', 'BAN2', 'BAYMINA', 'BILGIN1', 'BILGIN2', 'BURSA BLOK1', 'BURSA BLOK2', 'CENGIZ', 'ENKA ADP', 'ENKA GBZ1', 'ENKA GBZ2', 'ENKA IZM1', 'ENKA IZM2', 'GAMA ICAN', 'HABAS', 'HAM-10', 'HAM-20', 'RWE', 'TEKIRA', 'TEKIRB', 'YENI', 'IST A-(A)', 'IST A-(B)', 'IST A-(C)', 'IST B (Blok40+ Blok50)', 'year', 'month', 'day', 'day_of_week', 'is_weekend', 'hour_numeric', 'season']


In [37]:
# 📌 SANTRALLERİN ÇALIŞMA DURUMUNU TAHMİN ETMEK İÇİN VERİYİ HAZIRLA
X = electricity_data[features]

# 📌 SEÇİLEN SANTRALLER (Tahmin etmek istediğimiz rakipler)
selected_players = ["ACWA", "BAYMINA", "ENKA", "RWE"]  

# 📌 Y (Hedef Değişken) → Santralin Çalışıp Çalışmadığı (1 veya 0)
y = electricity_data[[f"{plant}_operational" for plant in selected_players]]

# 📌 TRAIN-TEST AYIRMA
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# 📌 SMOTE KULLANARAK VERİYİ DENGELEME
smote = SMOTE()
X_train_balanced, y_train_balanced = smote.fit_resample(X_train, y_train)

# 📌 RANDOM FOREST CLASSIFIER İLE MODELİ EĞİTME
rf_classifier = RandomForestClassifier(n_estimators=200, max_depth=10, random_state=42)
rf_classifier.fit(X_train_balanced, y_train_balanced)

# 📌 MODELİN TAHMİN SONUÇLARINI KAYDET
predictions = rf_classifier.predict(X_test)

for i, plant in enumerate(selected_players):
    X_test[f"{plant}_predicted"] = predictions[:, i]


KeyError: "None of [Index(['ACWA_operational', 'BAYMINA_operational', 'ENKA_operational',\n       'RWE_operational'],\n      dtype='object')] are in the [columns]"